In [ ]:
from qiniu import Auth, put_file, etag
import pandas as pd
import requests
import shutil
import time
import glob
import os

os.chdir('E:/photo')  # 图片保存的路径
file_path = 'E:/photo_path'  # 上传成功后保存的路径
timestamp = str(int(time.time()))  # 时间戳
access_key = '***'
secret_key = '***'  # 秘钥对
q = Auth(access_key, secret_key)  # 鉴权
bucket_name = '***'  # 存储空间名称
bucket_domain = '***'  # 空间外链
filenames = glob.glob('*.*')  # 获取文件夹内图片名称生成列表
result = pd.DataFrame(columns=('photo_name', 'photo_url'))  # 定义空的Dataframe保存内容

for key in filenames:
    # 生成上传 Token，指定过期时间
    token = q.upload_token(bucket_name, key, 3600)
    # 要上传图片的本地路径
    localfile = 'E:/photo/{}'.format(key)
    # 上传图片
    ret, info = put_file(token, key, localfile)
    print(info)
    assert ret['key'] == key
    assert ret['hash'] == etag(localfile)
    # 上传后将移动图片到file_path
    shutil.move(localfile, file_path)
    # 构造下载base_url
    base_url = 'http://%s/%s' % (bucket_domain, key)
    # 断言请求链接是否成功,否则触发异常
    r = requests.get(base_url)
    assert r.status_code == 200
    # 保存规格码和图片链接
    result = result.append(pd.DataFrame({'photo_name': [os.path.splitext(key)[
                           0]], 'photo_url': [base_url]}), ignore_index=True)

result.rename(columns={'photo_name': '图片名称',
                       'photo_url': '图片链接'}, inplace=True)  # 修改列名
os.chdir('C:/Users/Samuel/Desktop/url_data')  # 图片链接保存的路径
result.to_excel('图片链接' + timestamp + '.xlsx', encoding='utf-8',
                sheet_name='图片链接', index=False)  # 导出文件